In [16]:
from pyspark.mllib.recommendation import ALS
import math

# Some constants
DATASET_RATINGS_PATH = "./ratings_train.csv"
DATASET_MOVIES_PATH = "./movies.csv"

# Define some helper functions
def readCSV(fname, removeHeader=False, separator=','):
    print("Loading file ", fname, "...")
    rdd = sc.textFile(fname)
    if removeHeader:
        firstline = rdd.first()
        rdd = rdd.filter(lambda x: x != firstline)
    return rdd.map(lambda x: x.split(separator))

# Load the ratings data
ratings_data = readCSV(DATASET_RATINGS_PATH, removeHeader=True)

# Load the movies data
movies_data = readCSV(DATASET_MOVIES_PATH, removeHeader=True)

# Parse the ratings data
# [user_id, movie_id, rating, timestamp] -> (user_id, movie_id, rating)
ratings_data = ratings_data.map(lambda x: tuple(x[0].split('::')[:-1]))\
    .cache()
print("There are", ratings_data.count(), "ratings in the dataset.")

# Parse the movies data
# [id, title, genres[]] -> (id, title)
movies_data = movies_data.map(lambda x: tuple([x[0], x[1]]))\
    .cache()
print("There are", movies_data.count(), "movies in the dataset.")


Loading file  ./ratings_train.csv ...
Loading file  ./movies.csv ...
